# Remove host reads from virus bus file using flag

Align to host with -n column in bus file:

In [1]:
import glob

In [13]:
bp = "PRJNA665227"
fastq_folder = f"/home/laura/projects/virus-watch-data/{bp}/raw"
sample_batch_file = f"{fastq_folder}/batch.txt"

In [3]:
# host_fasta = "/home/laura/projects/virus-watch-data/rhesus_ref/Macaca_mulatta.Mmul_10.dna.toplevel.fa.gz"
# host_gtf = "/home/laura/projects/virus-watch-data/rhesus_ref/Macaca_mulatta.Mmul_10.109.gtf.gz"

# canine_fasta = "/home/laura/projects/virus-watch-data/canine_ref/Canis_lupus_familiaris.ROS_Cfam_1.0.dna.toplevel.fa.gz"
# canine_gtf = "/home/laura/projects/virus-watch-data/canine_ref/Canis_lupus_familiaris.ROS_Cfam_1.0.109.gtf.gz"

# canine_macaque_fasta = "/home/laura/projects/virus-watch-data/canine_macaque_ref/combined_genomes.dna.toplevel.fa.gz"

index = "/home/laura/projects/virus-watch-data/canine_macaque_ref/index.idx"
t2g = "/home/laura/projects/virus-watch-data/canine_macaque_ref/t2g.txt"
fa = "/home/laura/projects/virus-watch-data/canine_macaque_ref/transcriptome.fa"

kallisto = "/home/laura/temp/kallisto/build/src/kallisto"
bustools = "/home/laura/projects/bustools/build/src/bustools"

In [4]:
host_out_folder = f"{fastq_folder}/../host_flag/host"

In [5]:
# %%time
# !/usr/bin/time -v $kallisto bus \
#         -n \
#         -i $index \
#         -o $host_out_folder \
#         -t 40 \
#         -B $sample_batch_file \
#         --batch-barcodes \
#         -x 0,0,12:0,12,20:1,0,0

Align to virus with -n column in bus file + index masked with d-list cdna + dna:

In [6]:
out_folder = f"{fastq_folder}/../host_flag/virus_cdna_dna"

In [17]:
virus_fasta = "/home/laura/projects/virus-watch-data/virus_ref/uniques_noduplicates.fa"
virus_index = "/home/laura/projects/virus-watch-data/virus_ref/kallisto_aa/noduplicates_rhesus_canine_cdna_dna.idx"
virus_t2g = "/home/laura/projects/virus-watch-data/virus_ref/nodup_clu_t2g.txt"

In [18]:
%%time
!/usr/bin/time -v $kallisto bus \
        -n \
        -i $virus_index \
        -o $out_folder \
        --aa \
        -t 40 \
        -B $sample_batch_file \
        --batch-barcodes \
        -x 0,0,12:0,12,20:1,0,0


[bus] will try running read files supplied in batch file
[bus] Note: Strand option was not specified; setting it to --unstranded for specified technology
[index] k-mer length: 31
[index] number of targets: 296,561
[index] number of k-mers: 37,632,571
[index] number of distinguishing flanking k-mers: 91,841
[quant] running in single-end mode
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698516_1.fastq.gz
[quant] will process file 2: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698516_2.fastq.gz
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698524_1.fastq.gz
[quant] will process file 2: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698524_2.fastq.gz
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698536_1.fastq.gz
[quant] will process file 2: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698536_2.fastq.gz
[quant] will process

### Split virus matrix into reads seen in host and those not seen in host
This has to be done for each batch (SRR library) separately, since the read numbers reset for each batch:

In [19]:
from tqdm import tqdm

In [20]:
# Get list of sample barcodes
sb_file = f"{host_out_folder}/matrix.sample.barcodes"

with open(sb_file, "r") as sb:
    sample_barcodes = sb.read().splitlines()
    
sample_barcodes[:10]

['AAAAAAAAAAAAAAAA',
 'AAAAAAAAAAAAAAAC',
 'AAAAAAAAAAAAAAAG',
 'AAAAAAAAAAAAAAAT',
 'AAAAAAAAAAAAAACA',
 'AAAAAAAAAAAAAACC',
 'AAAAAAAAAAAAAACG',
 'AAAAAAAAAAAAAACT',
 'AAAAAAAAAAAAAAGA',
 'AAAAAAAAAAAAAAGC']

In [22]:
%%time
sample_barcodes_star = f"{host_out_folder}/matrix.sample.barcodes.tmp"

TQDM_BAR_FORMAT = (
    "{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]"
)

with tqdm(total=len(sample_barcodes), bar_format=TQDM_BAR_FORMAT) as pbar:
    for sb in sample_barcodes:
        with open(sample_barcodes_star, "w") as sbs_file:
            sbs_file.write(sb + "*")

        print(f"Capture reads for batch {sb} for virus and host alignments...")
        # Capture records for this batch from virus and host bus files
        !$bustools capture \
            $out_folder/output.bus \
            --capture $sample_barcodes_star \
            --barcode \
            -o $out_folder/output_sample_tmp.bus

        !$bustools capture \
            $host_out_folder/output.bus \
            --capture $sample_barcodes_star \
            --barcode \
            -o $host_out_folder/output_sample_tmp.bus

        print(f"Split viral alignments into host/masked for batch {sb}...")
        # Capture records from virus reads also mapped to host
        bus_name = f"output_{sb}_host.bus"
        !$bustools capture \
            $out_folder/output_sample_tmp.bus \
            --capture <($bustools text -pf $host_out_folder/output_sample_tmp.bus | cut -d$'\t' -f5)  \
            --flags \
            -o $out_folder/$bus_name

        # Capture records from virus reads that did not map to host
        bus_name = f"output_{sb}_virus.bus"
        !$bustools capture \
            $out_folder/output_sample_tmp.bus \
            --capture <($bustools text -pf $host_out_folder/output_sample_tmp.bus | cut -d$'\t' -f5)  \
            --complement \
            --flags \
            -o $out_folder/$bus_name
        
        pbar.update(n=1)

  0%|                                                                                                                           | 0/106 [elapsed: 00:00 remaining: ?]

Capture reads for batch AAAAAAAAAAAAAAAA for virus and host alignments...
Read in 236692562 BUS records, wrote 2120107 BUS records
Read in 8828381950 BUS records, wrote 90771029 BUS records
Split viral alignments into host/masked for batch AAAAAAAAAAAAAAAA...
Read in 90771029 BUS records
Read in 2120107 BUS records, wrote 1220 BUS records
Read in 90771029 BUS records
Read in 2120107 BUS records, wrote 2118887 BUS records


  1%|█                                                                                                                   | 1/106 [elapsed: 10:43 remaining: 18:46:45]

Capture reads for batch AAAAAAAAAAAAAAAC for virus and host alignments...
Read in 236692562 BUS records, wrote 470933 BUS records
Read in 8828381950 BUS records, wrote 68474890 BUS records
Split viral alignments into host/masked for batch AAAAAAAAAAAAAAAC...
Read in 68474890 BUS records
Read in 470933 BUS records, wrote 1596 BUS records
Read in 68474890 BUS records
Read in 470933 BUS records, wrote 469337 BUS records


  2%|██▏                                                                                                                 | 2/106 [elapsed: 20:57 remaining: 18:05:02]

...

Capture reads for batch AAAAAAAAAAAACGGA for virus and host alignments...
Read in 236692562 BUS records, wrote 855724 BUS records
Read in 8828381950 BUS records, wrote 30617474 BUS records
Split viral alignments into host/masked for batch AAAAAAAAAAAACGGA...
Read in 30617474 BUS records
Read in 855724 BUS records, wrote 639 BUS records
Read in 30617474 BUS records
Read in 855724 BUS records, wrote 855085 BUS records


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 105/106 [elapsed: 19:22:22 remaining: 11:05]

Capture reads for batch AAAAAAAAAAAACGGC for virus and host alignments...
Read in 236692562 BUS records, wrote 1172956 BUS records
Read in 8828381950 BUS records, wrote 38488143 BUS records
Split viral alignments into host/masked for batch AAAAAAAAAAAACGGC...
Read in 38488143 BUS records
Read in 1172956 BUS records, wrote 549 BUS records
Read in 38488143 BUS records
Read in 1172956 BUS records, wrote 1172407 BUS records


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [elapsed: 19:32:58 remaining: 00:00]

CPU times: user 22min 48s, sys: 5min 3s, total: 27min 51s
Wall time: 19h 32min 58s


### Clean up cell barcodes using host cell whitelist generated by bustools

In [23]:
host_whitelist_folder = f"/home/laura/projects/virus-watch-data/{bp}/full_index_stringent_dlist_flank1/host_canine"

In [24]:
import os

In [25]:
%%time
with tqdm(total=len(sample_barcodes), bar_format=TQDM_BAR_FORMAT) as pbar:
    for sb in sample_barcodes:
        for ent in ["host", "virus"]:
            bus_name = f"output_{sb}_{ent}.bus"
            
            # Correct barcodes (no need to sort first)
            # This needs to be done before removing the flag column since adding the sample barcodes to the cell barcodes will confuse bustools correct
            !$bustools correct \
                -w $host_whitelist_folder/bustools_whitelist.txt \
                -o $out_folder/output_temp1.bus \
                $out_folder/$bus_name

            # Remove flag column so bustools count does not get confused
            # -a retains the sample barcodes and adds them to the normal cell barcodes
            !$bustools text \
                -apf $out_folder/output_temp1.bus | cut -d$'\t' -f1,2,3,4 \
                | $bustools fromtext -o $out_folder/output_temp2.bus -

            # Sort and count
            !$bustools sort \
                -m 4G \
                -t 30 \
                -o $out_folder/output_temp3.bus \
                $out_folder/output_temp2.bus

            os.makedirs(f"{out_folder}/{sb}/{ent}", exist_ok=True)
            !$bustools count \
                --genecounts \
                -o $out_folder/$sb/$ent/bustools_count/ \
                -g $virus_t2g \
                -e $out_folder/matrix.ec \
                -t $out_folder/transcripts.txt \
                $out_folder/output_temp3.bus
        
        pbar.update(n=1)

  0%|                                                                                                                           | 0/106 [elapsed: 00:00 remaining: ?]

Found 775516 barcodes in the on-list
Processed 1220 BUS records
In on-list = 903
Corrected    = 90
Uncorrected  = 227
Read in 993 BUS records
Read in 993 text records
 all fits in buffer
Read in 993 BUS records
reading time 0s
sorting time 0s
writing time 0s
Found 775516 barcodes in the on-list
Processed 2118887 BUS records
In on-list = 246632
Corrected    = 566522
Uncorrected  = 1305733
Read in 813154 BUS records
Read in 813154 text records
partition time: 0.03s
 all fits in buffer
Read in 813154 BUS records
reading time 0.01s
sorting time 0.5s
writing time 0.02s


  1%|█                                                                                                                      | 1/106 [elapsed: 00:18 remaining: 32:47]

Found 775516 barcodes in the on-list
Processed 1596 BUS records
In on-list = 973
Corrected    = 184
Uncorrected  = 439
Read in 1157 BUS records
Read in 1157 text records
 all fits in buffer
Read in 1157 BUS records
reading time 0s
sorting time 0s
writing time 0s
Found 775516 barcodes in the on-list
Processed 469337 BUS records
In on-list = 73108
Corrected    = 114247
Uncorrected  = 281982
Read in 187355 BUS records
Read in 187355 text records
partition time: 0.01s
 all fits in buffer
Read in 187355 BUS records
reading time 0s
sorting time 0.08s
writing time 0s


  2%|██▏                                                                                                                    | 2/106 [elapsed: 00:33 remaining: 28:18]

...

Found 775516 barcodes in the on-list
Processed 639 BUS records
In on-list = 331
Corrected    = 107
Uncorrected  = 201
Read in 438 BUS records
Read in 438 text records
 all fits in buffer
Read in 438 BUS records
reading time 0s
sorting time 0s
writing time 0s
Found 775516 barcodes in the on-list
Processed 855085 BUS records
In on-list = 102272
Corrected    = 223308
Uncorrected  = 529505
Read in 325580 BUS records
Read in 325580 text records
partition time: 0.01s
 all fits in buffer
Read in 325580 BUS records
reading time 0.01s
sorting time 0.11s
writing time 0.01s


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 105/106 [elapsed: 33:16 remaining: 00:17]

Found 775516 barcodes in the on-list
Processed 549 BUS records
In on-list = 344
Corrected    = 67
Uncorrected  = 138
Read in 411 BUS records
Read in 411 text records
 all fits in buffer
Read in 411 BUS records
reading time 0s
sorting time 0s
writing time 0.01s
Found 775516 barcodes in the on-list
Processed 1172407 BUS records
In on-list = 168432
Corrected    = 294007
Uncorrected  = 709968
Read in 462439 BUS records
Read in 462439 text records
partition time: 0.03s
 all fits in buffer
Read in 462439 BUS records
reading time 0.01s
sorting time 0.45s
writing time 0.01s


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [elapsed: 33:34 remaining: 00:00]

CPU times: user 39.2 s, sys: 14 s, total: 53.2 s
Wall time: 33min 34s
